<a href="https://colab.research.google.com/github/OtolaHan/Otola_Han_KAAN/blob/main/utitls.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import os
import pandas as pd
from sklearn.linear_model import LinearRegression
from utils import create_folder, smape  # Импортируем функции из utils.py


def load_data(file_path):
    return pd.read_csv(file_path, parse_dates=['datetime'], index_col='datetime')


def train_model(data):
    X = data[:-100].index.astype(int).values.reshape(-1, 1)  # Используем время как признак
    y = data['ssh'][:-100].values
    model = LinearRegression()
    model.fit(X, y)
    return model


def generate_predictions(model, start_time, num_predictions):
    prediction_times = pd.date_range(start=start_time, periods=num_predictions, freq='H')
    X_pred = prediction_times.astype(int).values.reshape(-1, 1)
    predictions = model.predict(X_pred)
    return pd.DataFrame({'ssh': predictions}, index=prediction_times)


def save_predictions(predictions, file_name, save_folder):
    predictions.reset_index().rename(columns={'index': 'datetime'}).to_csv(os.path.join(save_folder, file_name),
                                                                           index=False)


def main():
    train_folder = "train"
    predictions_folder = "predictions"
    create_folder(predictions_folder)

    for file_name in os.listdir(train_folder):
        file_path = os.path.join(train_folder, file_name)
        data = load_data(file_path)

        model = train_model(data)

        start_time = '2002-12-26 21:00:00'
        num_predictions = 100
        predictions = generate_predictions(model, start_time, num_predictions)

        save_predictions(predictions, file_name, predictions_folder)


if __name__ == "__main__":
    main()